# Testing AutoDiff for functions in shortwave radiation

Here, I test how JAX VJP works with functions in the shortwave radiation fluxes file

In [1]:
import jax
import jax.numpy as jnp
from jax import grad
from jax import vjp
from jcm.model import initialize_modules

In [2]:
# Defining input parameters and parameter dependencies 
ix = 4         # Number of longitudes
iy = 3         # Number of latitudes in hemisphere
il = 2 * iy     # Number of latitudes in full sphere
kx = 5          # Number of vertical levels
tyear = 0.25  # Example time of the year (spring equinox)
xy = (ix, il)
xyz = (ix, il, kx)
initialize_modules(kx = kx, il = il)

In [ ]:
from jcm.physics import PhysicsTendency, PhysicsState
from jcm.physics_data import LWRadiationData, SWRadiationData, CondensationData, ConvectionData, HumidityData, SurfaceFluxData, DateData, PhysicsData
from jcm.shortwave_radiation import clouds, get_zonal_average_fields, solar

In [5]:
date_data = DateData(tyear=tyear)  # Define date data object
physics_data = PhysicsData.zeros(xy,kx,date=date_data)  # Create PhysicsData object (parameter)
#state = PhysicsState(jnp.zeros(xyz), jnp.zeros(xyz), jnp.zeros(xyz), jnp.zeros(xyz), jnp.zeros(xyz), jnp.zeros(xy)) # Create PhysicsState object (parameter)
state =PhysicsState.zeros(xyz)

In [6]:
#Test to see if function is working properly
get_zonal_average_fields(state, physics_data)

(PhysicsTendency(u_wind=Array([[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],
 
        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],
 
        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],
 
        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]], dtype=float32), v_wind=Array([[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],
 
        [[0

In [7]:
#Define gradient function

def grad_get_zonal_average_fields(f, state, physics_data): 
    '''
    Calculate the gradient of get_zonal_average_fields with respect to physics state and physics data
    '''
    primals, f_vjp = vjp(f, state, physics_data) 
    tends = primals[0].copy(jnp.ones_like(primals[0].u_wind),jnp.ones_like(primals[0].v_wind),
                            jnp.ones_like(primals[0].temperature),jnp.ones_like(primals[0].specific_humidity))
    datas = primals[1].copy()  #Note: would like to include a ones function to get accurate gradients
    input = (tends, datas)
    df_dtends, df_ddatas = f_vjp(input)
    return df_dtends, df_ddatas


In [8]:
df_dtends, df_ddatas = grad_get_zonal_average_fields(get_zonal_average_fields, state, physics_data)
print(df_dtends)
print(df_ddatas)


PhysicsState(u_wind=Array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]], dtype=float32), v_wind=Array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 